# 인공신경망 딥러닝 모델 구현

In [1]:
import tensorflow
tensorflow.__version__

'2.13.0'

## iris 데이터 불러오기

In [2]:
import pandas as pd

In [3]:
from sklearn import datasets

In [4]:
iris = datasets.load_iris()

In [5]:
iris_X = iris.data

In [6]:
iris_y = pd.get_dummies(iris.target).to_numpy()

In [7]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(iris_X, iris_y, test_size=0.3,
                                                    random_state=1)

## 싸이킷런을 이용한 구현

In [8]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(50,30), activation="logistic", solver="adam",
                    max_iter=1000) 

In [9]:
mlp.fit(train_X, train_y)

MLPClassifier(activation='logistic', hidden_layer_sizes=(50, 30), max_iter=1000)

In [10]:
mlp.score(test_X, test_y)

1.0

In [11]:
pred = mlp.predict(test_X)

In [12]:
import numpy as np
pd.crosstab(np.argmax(pred, axis=1), np.argmax(test_y, axis=1)) #교차분류표

col_0,0,1,2
row_0,,,
0,14,0,0
1,0,18,0
2,0,0,13


## 텐서플로우를 이용한 구현

In [13]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [14]:
x = tf.placeholder(tf.float32, [None, 4])
y = tf.placeholder(tf.float32, [None, 3])

In [15]:
W1 = tf.Variable(tf.zeros([4, 50]))
b1 = tf.Variable(tf.zeros([50]))
h1 = tf.nn.sigmoid(tf.matmul(x, W1)+b1)

In [17]:
W2 = tf.Variable(tf.zeros([50, 30]))
b2 = tf.Variable(tf.zeros([30]))
h2 = tf.nn.sigmoid(tf.matmul(h1, W2)+b2)

In [18]:
W3 = tf.Variable(tf.zeros([30, 3]))
b3 = tf.Variable(tf.zeros([3]))
h3 = tf.nn.softmax(tf.matmul(h2, W3)+b3)

In [19]:
cross_entropy = -tf.reduce_sum(y*tf.log(h3), reduction_indices=[1])
loss = tf.reduce_mean(cross_entropy)
train = tf.train.AdamOptimizer().minimize(loss)

In [20]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for i in range(1000):
    sess.run(train, feed_dict={x:train_X, y:train_y})
    if i%100 == 0:
        tr_loss = sess.run(loss, feed_dict={x:train_X, y:train_y})
        print(i, tr_loss)

0 1.0978113
100 1.0611217
200 0.76452047
300 0.6687561
400 0.6138217
500 0.56832063
600 0.5274489
700 0.4911589
800 0.45574504
900 0.41874182


In [21]:
correct_prediction = tf.equal(tf.argmax(h3, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x:test_X, y:test_y}))

0.8888889


In [22]:
sess.close()

## 케라스를 이용한 구현

In [23]:
from tensorflow.keras.models import Sequential
model = Sequential()

In [24]:
from tensorflow.keras.layers import InputLayer, Dense
model.add(InputLayer(4))
model.add(Dense(50, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [25]:
model.compile(optimizer='adam', loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [26]:
model.fit(train_X, train_y, epochs=1000)

Train on 105 samples
Epoch 1/1000
105/105 [==============================] - 0s 581us/sample - loss: 1.2979 - acc: 0.3524
Epoch 2/1000
105/105 [==============================] - 0s 52us/sample - loss: 1.1028 - acc: 0.3524
Epoch 3/1000
105/105 [==============================] - 0s 42us/sample - loss: 0.9992 - acc: 0.3524
Epoch 4/1000
105/105 [==============================] - 0s 52us/sample - loss: 0.9206 - acc: 0.3905
Epoch 5/1000
105/105 [==============================] - 0s 47us/sample - loss: 0.8626 - acc: 0.4571
Epoch 6/1000
105/105 [==============================] - 0s 49us/sample - loss: 0.8191 - acc: 0.7048
Epoch 7/1000
105/105 [==============================] - 0s 52us/sample - loss: 0.7779 - acc: 0.7810
Epoch 8/1000
105/105 [==============================] - 0s 50us/sample - loss: 0.7341 - acc: 0.8095
Epoch 9/1000
105/105 [==============================] - 0s 44us/sample - loss: 0.6972 - acc: 0.7905
Epoch 10/1000
105/105 [==============================] - 0s 44us/sample - loss

Epoch 82/1000
105/105 [==============================] - 0s 31us/sample - loss: 0.1321 - acc: 0.9810
Epoch 83/1000
105/105 [==============================] - 0s 50us/sample - loss: 0.1414 - acc: 0.9714
Epoch 84/1000
105/105 [==============================] - 0s 46us/sample - loss: 0.1355 - acc: 0.9714
Epoch 85/1000
105/105 [==============================] - 0s 41us/sample - loss: 0.1385 - acc: 0.9619
Epoch 86/1000
105/105 [==============================] - 0s 45us/sample - loss: 0.1332 - acc: 0.9619
Epoch 87/1000
105/105 [==============================] - 0s 42us/sample - loss: 0.1245 - acc: 0.9714
Epoch 88/1000
105/105 [==============================] - 0s 33us/sample - loss: 0.1447 - acc: 0.9619
Epoch 89/1000
105/105 [==============================] - 0s 35us/sample - loss: 0.1362 - acc: 0.9714
Epoch 90/1000
105/105 [==============================] - 0s 47us/sample - loss: 0.1344 - acc: 0.9524
Epoch 91/1000
105/105 [==============================] - 0s 45us/sample - loss: 0.1254 - ac

105/105 [==============================] - 0s 34us/sample - loss: 0.0945 - acc: 0.9619
Epoch 243/1000
105/105 [==============================] - 0s 36us/sample - loss: 0.0820 - acc: 0.9714
Epoch 244/1000
105/105 [==============================] - 0s 36us/sample - loss: 0.0849 - acc: 0.9714
Epoch 245/1000
105/105 [==============================] - 0s 43us/sample - loss: 0.0852 - acc: 0.9714
Epoch 246/1000
105/105 [==============================] - 0s 49us/sample - loss: 0.0807 - acc: 0.9714
Epoch 247/1000
105/105 [==============================] - 0s 42us/sample - loss: 0.0823 - acc: 0.9524
Epoch 248/1000
105/105 [==============================] - 0s 47us/sample - loss: 0.1025 - acc: 0.9429
Epoch 249/1000
105/105 [==============================] - 0s 32us/sample - loss: 0.1016 - acc: 0.9619
Epoch 250/1000
105/105 [==============================] - 0s 44us/sample - loss: 0.0900 - acc: 0.9524
Epoch 251/1000
105/105 [==============================] - 0s 46us/sample - loss: 0.0864 - acc: 0.

105/105 [==============================] - 0s 29us/sample - loss: 0.0763 - acc: 0.9714
Epoch 403/1000
105/105 [==============================] - 0s 43us/sample - loss: 0.0736 - acc: 0.9810
Epoch 404/1000
105/105 [==============================] - 0s 34us/sample - loss: 0.0781 - acc: 0.9714
Epoch 405/1000
105/105 [==============================] - 0s 38us/sample - loss: 0.0807 - acc: 0.9714
Epoch 406/1000
105/105 [==============================] - 0s 29us/sample - loss: 0.0751 - acc: 0.9810
Epoch 407/1000
105/105 [==============================] - 0s 36us/sample - loss: 0.0796 - acc: 0.9619
Epoch 408/1000
105/105 [==============================] - 0s 38us/sample - loss: 0.0846 - acc: 0.9524
Epoch 409/1000
105/105 [==============================] - 0s 35us/sample - loss: 0.0772 - acc: 0.9714
Epoch 410/1000
105/105 [==============================] - 0s 37us/sample - loss: 0.0740 - acc: 0.9810
Epoch 411/1000
105/105 [==============================] - 0s 32us/sample - loss: 0.0786 - acc: 0.

105/105 [==============================] - 0s 35us/sample - loss: 0.0824 - acc: 0.9524
Epoch 563/1000
105/105 [==============================] - 0s 41us/sample - loss: 0.0836 - acc: 0.9619
Epoch 564/1000
105/105 [==============================] - 0s 28us/sample - loss: 0.0763 - acc: 0.9714
Epoch 565/1000
105/105 [==============================] - 0s 23us/sample - loss: 0.0772 - acc: 0.9714
Epoch 566/1000
105/105 [==============================] - 0s 38us/sample - loss: 0.0711 - acc: 0.9810
Epoch 567/1000
105/105 [==============================] - 0s 36us/sample - loss: 0.0767 - acc: 0.9714
Epoch 568/1000
105/105 [==============================] - 0s 34us/sample - loss: 0.0753 - acc: 0.9714
Epoch 569/1000
105/105 [==============================] - 0s 38us/sample - loss: 0.0721 - acc: 0.9714
Epoch 570/1000
105/105 [==============================] - 0s 33us/sample - loss: 0.0735 - acc: 0.9714
Epoch 571/1000
105/105 [==============================] - 0s 30us/sample - loss: 0.0723 - acc: 0.

105/105 [==============================] - 0s 38us/sample - loss: 0.0703 - acc: 0.9714
Epoch 723/1000
105/105 [==============================] - 0s 33us/sample - loss: 0.0709 - acc: 0.9714
Epoch 724/1000
105/105 [==============================] - 0s 37us/sample - loss: 0.0729 - acc: 0.9714
Epoch 725/1000
105/105 [==============================] - 0s 38us/sample - loss: 0.0707 - acc: 0.9714
Epoch 726/1000
105/105 [==============================] - 0s 38us/sample - loss: 0.0704 - acc: 0.9810
Epoch 727/1000
105/105 [==============================] - 0s 39us/sample - loss: 0.0712 - acc: 0.9810
Epoch 728/1000
105/105 [==============================] - 0s 37us/sample - loss: 0.0712 - acc: 0.9810
Epoch 729/1000
105/105 [==============================] - 0s 39us/sample - loss: 0.0718 - acc: 0.9810
Epoch 730/1000
105/105 [==============================] - 0s 46us/sample - loss: 0.0706 - acc: 0.9714
Epoch 731/1000
105/105 [==============================] - 0s 39us/sample - loss: 0.0716 - acc: 0.

105/105 [==============================] - 0s 37us/sample - loss: 0.0741 - acc: 0.9714
Epoch 883/1000
105/105 [==============================] - 0s 36us/sample - loss: 0.0725 - acc: 0.9619
Epoch 884/1000
105/105 [==============================] - 0s 31us/sample - loss: 0.0698 - acc: 0.9714
Epoch 885/1000
105/105 [==============================] - 0s 46us/sample - loss: 0.0699 - acc: 0.9810
Epoch 886/1000
105/105 [==============================] - 0s 36us/sample - loss: 0.0740 - acc: 0.9714
Epoch 887/1000
105/105 [==============================] - 0s 33us/sample - loss: 0.0727 - acc: 0.9810
Epoch 888/1000
105/105 [==============================] - 0s 44us/sample - loss: 0.0716 - acc: 0.9810
Epoch 889/1000
105/105 [==============================] - 0s 31us/sample - loss: 0.0702 - acc: 0.9714
Epoch 890/1000
105/105 [==============================] - 0s 29us/sample - loss: 0.0682 - acc: 0.9714
Epoch 891/1000
105/105 [==============================] - 0s 33us/sample - loss: 0.0718 - acc: 0.

In [27]:
model.evaluate(test_X, test_y)

C:\Users\kosa\anaconda3\lib\site-packages\keras\src\engine\training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


[0.024779820835424793, 1.0]

In [ ]:
# 위 출력값은 loss와 accuracy입니다.